Sample code to reduce the outliers in a model.

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default='iframe'
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import os
os.environ["HUGGINGFACE_CO_TIMEOUT"] = "600"  # Increase timeout duration to 600 seconds
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import hdbscan
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=8, prediction_data = True)
from umap import UMAP
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)

MODEL Reduced outliers using c-tfidf and distribution strategy.

Group 4 as example. This can be rewritten to iterate over all the models.

In [ ]:
group4_df = pd.read_csv('./datasets/group4_spanien.csv')

docs = group4_df['text']
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True, reduce_frequent_words=True)
group4_topic_model = BERTopic(language='danish', calculate_probabilities=True, umap_model=umap_model, hdbscan_model=hdbscan_model, embedding_model="paraphrase-multilingual-MiniLM-L12-v2", top_n_words=10, min_topic_size=10, ctfidf_model=ctfidf_model, verbose=True)
topics, probs = group4_topic_model.fit_transform(docs, embeddings)
# Use the "c-TF-IDF" strategy with a threshold
new_topics = group4_topic_model.reduce_outliers(docs, topics , strategy="c-tf-idf", threshold=0.1)

# Reduce all outliers that are left with the "distributions" strategy
new_topics = group4_topic_model.reduce_outliers(docs, topics, strategy="distributions")
group4_updated_dists = group4_topic_model
group4_updated_dists.update_topics(docs, topics=new_topics)

Save the model and embeddings. 

In [ ]:
group4_topic_model.save('./reduced_models/group4_topic_model_0720')
np.save('./reduced_models/group4_embeddings_0720.npy', embeddings)